<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

Mounted at /content.gdrive


In [4]:
import json
import numpy as np

#merging json files
with open('/content.gdrive/MyDrive/Datasets/data_manually_labeled_original.json', "r") as fp:
    data1 = json.load(fp)

# convert lists to numpy arrays
X1 = np.array(data1["keypoints"])
#X = np.array(data["mfcc"])
y1 = np.array(data1["labels"])

with open('/content.gdrive/MyDrive/Datasets/data_manually_labled_Telemark_mirror.json', "r") as fp:
    data2 = json.load(fp)
# convert lists to numpy arrays
X2 = np.array(data2["keypoints"])
#X = np.array(data["mfcc"])
y2 = np.array(data2["labels"])

X = np.concatenate((X1,X2), axis=0)
y = np.concatenate((y1,y2), axis=0)
print(X[0])
print(X[332])
print(X.shape)
print(y[0])
print(y[332])
print(y.shape)

[0.71802741 0.29048792 0.24816319 ... 0.35894373 0.65924037 0.42562544]
[0. 0. 0. ... 0. 0. 0.]
(443, 1575)
1
1
(443,)


In [ ]:
#test load data
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

with open('/content.gdrive/MyDrive/Manually_Labeled/data.json', "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
X = np.array(data["keypoints"])
#X = np.array(data["mfcc"])
y = np.array(data["labels"])

print(X)
print(X.shape)
print(y)
print(y.shape)
"""
k = 0
for i in range(332):
    count = 0 
    for y in range(1575):
        if X[i][y] == 0:
            count = count +1
    #if count > 350:
        #k = k + 1
    print(count)        

print(k)
"""

[[0.71802741 0.29048792 0.24816319 ... 0.35894373 0.65924037 0.42562544]
 [0.68126655 0.28763264 0.38451716 ... 0.39175215 0.70825779 0.4498418 ]
 [0.70735633 0.29627445 0.15694906 ... 0.41513729 0.74908066 0.61638552]
 ...
 [0.21512574 0.36664426 0.2706992  ... 0.49334985 0.64569932 0.10506787]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.27345929 0.35210636 0.273204   ... 0.61386955 0.70903111 0.57472718]]
(332, 1575)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

'\nk = 0\nfor i in range(332):\n    count = 0 \n    for y in range(1575):\n        if X[i][y] == 0:\n            count = count +1\n    #if count > 350:\n        #k = k + 1\n    print(count)        \n\nprint(k)\n'

In [23]:
#Creating binary classification model
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.utils import shuffle


model = Sequential([
    Dense(units=2500, input_shape=(1575,), activation='relu'),
    Dense(units=1750, activation='relu'),
    Dense(units=1250, activation='relu'),
    Dense(units=750, activation='relu'),
    Dense(units=250, activation='relu'),
    Dense(units=2, activation='softmax')
])

model.summary()



model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 2500)              3940000   
                                                                 
 dense_31 (Dense)            (None, 1750)              4376750   
                                                                 
 dense_32 (Dense)            (None, 1250)              2188750   
                                                                 
 dense_33 (Dense)            (None, 750)               938250    
                                                                 
 dense_34 (Dense)            (None, 250)               187750    
                                                                 
 dense_35 (Dense)            (None, 2)                 502       
                                                                 
Total params: 11,632,002
Trainable params: 11,632,002


In [24]:
model.fit(x=X, y=y, batch_size=10, validation_split = 0.2, epochs=19, shuffle=True, verbose = 2)

Epoch 1/19
36/36 - 4s - loss: 0.6756 - accuracy: 0.5960 - val_loss: 1.0054 - val_accuracy: 0.0337 - 4s/epoch - 99ms/step
Epoch 2/19
36/36 - 3s - loss: 0.6411 - accuracy: 0.6525 - val_loss: 0.9685 - val_accuracy: 0.1573 - 3s/epoch - 78ms/step
Epoch 3/19
36/36 - 3s - loss: 0.5903 - accuracy: 0.6864 - val_loss: 0.5412 - val_accuracy: 0.7528 - 3s/epoch - 82ms/step
Epoch 4/19
36/36 - 3s - loss: 0.4993 - accuracy: 0.7599 - val_loss: 0.7959 - val_accuracy: 0.5169 - 3s/epoch - 79ms/step
Epoch 5/19
36/36 - 3s - loss: 0.4623 - accuracy: 0.7627 - val_loss: 1.2557 - val_accuracy: 0.2135 - 3s/epoch - 82ms/step
Epoch 6/19
36/36 - 3s - loss: 0.3885 - accuracy: 0.8249 - val_loss: 0.3784 - val_accuracy: 0.8315 - 3s/epoch - 80ms/step
Epoch 7/19
36/36 - 3s - loss: 0.3225 - accuracy: 0.8701 - val_loss: 0.2260 - val_accuracy: 0.8539 - 3s/epoch - 79ms/step
Epoch 8/19
36/36 - 3s - loss: 0.2731 - accuracy: 0.8927 - val_loss: 0.0544 - val_accuracy: 0.9888 - 3s/epoch - 78ms/step
Epoch 9/19
36/36 - 3s - loss: 0.

In [25]:
with open('/content.gdrive/MyDrive/Datasets/data_manually_labled_Telemark_mirror.json', "r") as fp:
    data2 = json.load(fp)
# convert lists to numpy arrays
X2 = np.array(data2["keypoints"])
#X = np.array(data["mfcc"])
y2 = np.array(data2["labels"])


predictions = model.predict(X2, batch_size = 10, verbose =0)

for i in predictions:
  if i[1] < 0.5:
      print("wrong")


wrong
wrong
